In [ ]:
'''Prepare the data and the model'''
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
'''Loading Dataset'''
training_data = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    transform = ToTensor()
)
test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor()
)

'''Preparing your data for training with DataLoaders'''
from torch.utils.data import DataLoader
train_dataloader = DataLoader(training_data, batch_size = 64, shuffle = True)
test_dataloader = DataLoader(test_data, batch_size = 64, shuffle = True)

In [ ]:
'''Define the Model'''
from torch import nn
class NeuralNetwork(nn.Module): #自定义的神经网络必须要subclass nn.Module
    def __init__(self):
        # 在__init__函数中初始化神经网络的layers
        super().__init__() #千万不要忘记调用父类的__init__()函数，注册所有的参数，并且训练过程中才能进行梯度的计算和更新
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        # 在forward函数中就是将输入走一遍我们在__init__()中定义的网络结构，得到最后的输出
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
# 注意：
# 1.不要直接call model.forward()!

In [ ]:
'''Hyperparameters'''
learning_rate = 1e-3 #在每个batch/epoch中更新模型参数的程度.学习率越小，学习的速度越慢，越大可能会导致训练过程中未知的行为。
batch_size = 64 #在参数更新前，在网络中传播的数据样本个数
epochs = 5 #迭代整个数据集的次数

# 1.常用的学习率设置策略
## 1.1 手动设置
就是直接在初始化optimizer的时候，手动设置lr。

## 1.2 学习率Warm-up策略
### 1.2.1 LambdaLR
（1）概念：线性地将学习率从一个很小的值，增大到目标值。
（2）用法：
```python
    from torch.optim.lr_scheduler import LambdaLR

    def warmup(epoch):
        return min(1.0, epoch / 5) #在前5个epochs线性增长
    
    scheduler = LambdaLR(optimizer, lr_lambda = warmup) 

    #背后的作用机制：
    #假设在optimizer中初始化的lr是0.01，那么会将warmup函数的返回值，作为缩放因子，乘到lr上，因此根据warmup函数的定义，在前5个epoch，lr分别为：0.002,0.004,0.006,0.008, 0.010。在此之后的lr都是0.01
```
（3）适用场景：对于Transformer-based模型。

### 1.2.2 OneCycleLR
（1）概念：学习率先上升，然后紧接着伴随cosine曲线式的下降。
（2）用法：
```python
    from torch.optim.lr_scheduler import OneCycleLR
    schduler = OneCycleLR(optimizer, max_lr = 0.1, total_steps = 100)
    #作用机制：
    # 使用OneCycleLR之后，在optimizer中设置的lr会被忽略，转而OneCycleLR会动态地调整
    #学习率，从一个很小的随机值，逐渐上升到max_lr，然后再下降到一个很小的随机值。这个过程会在total_steps中完成，所以total_steps一般设置为训练的epochs数。
```
（3）适用场景：大数据集，大学习率的实验。

## 1.3 学习率Decay的策略
### 1.3.1 StepLR
（1）概念：每隔固定的step_size，学习率都会乘上一个缩放因子gamma。
（2）用法：
```python
    from torch.optim.lr_scheduler import StepLR
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)
    scheduler = StepLR(optimizer, step_size = 10, gamma = 0.1)
    #作用机制：
    # 每隔step_size，lr都会乘上一个gamma
```
### 1.3.2 其他常用的学习率衰减策略
![](./images/学习率衰减策略.jpg)

## 1.4 自适应学习率策略-ReduceLROnPlateau
（1）介绍：会根据validation loss和模型性能改变学习率
（2）用法：
```python
    from torch.optim.lr_scheduler import ReduceLROnPlateau

    scheduler = ReduceLROnPlateau(optimzer, model = "min", patience = 5, factor = 0.1)

    #during training :
    optimizer.step()
    scheduler.step(val_loss) #需要把监视的指标穿进去，这里假设监视的是validation loss

    #作用机制：
    # ReduceLROnPlateau会监视val_loss的变换，如果连续patience轮都没有变换，那么就会将lr乘以factor
```

In [ ]:
'''Loss Function'''
loss = nn.CrossEntropyLoss()

# 1.loss函数介绍
## 1.1 常用的loss函数
nn.MSELoss(Mean Square Error): 均方误差，通常用于回归任务<br>
nn.NLLLoss(Negative Log Likelihood):负对数似然，用于分类任务<br>
nn.CrossEntropyLoss结合了nn.LogSoftmax和nn.NLLLoss<br>
![](./images/常用的loss函数.jpg)

## 1.2 torch.nn中常用的loss函数介绍
### 1.2.1 nn.MSELoss()
（1）定义：计算预测值和真实值之间的平方差的均值。<br>

（2）公式：<br>
![](./images/MSELoss的公式.png)<br>

（3）几何意义：假设将预测值和真实值想象成二维空间中的两个点，那么MSELoss就是在计算两个点之间距离的平方的均值。最小化MSE，就等价于最小化预测点和真实点之间的距离。<br>

（4）特点：<br>
MSE对于离群点是很敏感的，因为在公式中，距离被平方了，所以如果有距离很大的离群点，那么会对整个loss的计算带来很大的影响。<br>

（5）适用情况：通常用于回归任务，就是要预测房价，股价这种连续数值的任务。<br>

### 1.2.2 nn.L1Loss()
（1）定义：<br>
计算预测值和真实值之间的绝对距离的均值。又叫做MAE Loss。<br>

（2）数学公式：<br>
![](./images/L1Loss的公式.png)<br>

（3）特点：<br>
输入的shape是（N，* \)，*指的是任意其他维度。输出的shape和输入一样，只不过除了batch维，就是真实值。<br>
非负性质，L1Loss >= 0<br>
相较于MSE Loss，MAE Loss对于离群点更加robust，因为它没有平方。<br>
不平滑：在L1Loss = 0的地方是非平滑的，有可能导致在计算梯度的时候出现问题。<br>

（4）适用情况：<br>
与MSE Loss一样，适用于回归问题。<br>
模型的Regulation项。通常在Loss中加一个L1Loss作为模型的惩罚项。主要作用是防止过拟合，对离群点鲁棒。<br>

### 1.2.3 nn.BCEWithLogitsLoss()
(1)数学推导：
使用sigmoid激活函数，将输出的logits中的值转换为概率：<br>
![](./images/BCELoss的公式.png)<br>

得到输出的概率之后，就可以使用BCE Loss来计算了：<br>
![](./images/BCELoss的公式_1.png)<br>

(2)特点：<br>
接受输入的形状为：(N):single-label 二分类问题；（N, C）multi-label 二分类问题。C是label的个数，label的取值仍然是0或者1。这些输入也都是模型直接输出的logits，而不能经过Softmax或者sigmoid等激活函数的处理，因为BCELoss内置了sigmoid函数的处理。输出的形状和输入是一样的，但是值为0-1标签。<br>

（3）适用情况：<br>
single label的二元分类：比如垃圾邮件检测问题，模型对于一个sample只输出包含一个元素的logit。<br>
multi label的二元分类：比如图像打标签问题，一张图片可以同时被打上多个tags，（e.g., an image can be tagged as "cat", "animal", and "outdoor" simultaneously），每一个tag表示一个二元分类问题。模型输出的logits中对于每个tag都有一个元素值。<br>

（4）需要Sigmoid函数先处理——nn.BCELoss()<br>
![](./images/BCEloss的介绍.jpg)

### 1.2.4 nn.CrossEntropy()
(1)数学推导：<br>
![](./images/CrossEntropyLoss的数学推导.png)<br>

(2)特点：<br>
它计算了预测的概率分布和真实分布之间的差距。通过数学推导可以看出，loss鼓励模型对于正确的分类输出更高的概率。<br>
结合了nn.LogSoftmax()和nn.NLLLoss()，内置了softmax和负对数似然操作。<br>
它接受(batch, class)形状的输入logits，没有经过任何处理，特别是softmax。输出的结果形状是（batch），包含了这个batch中每个sample对应类别的下标。<br>

(3)适用情况，从推导过程来看，非常适合多分类任务。<br>


In [ ]:
'''Optimizer'''
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

# 1.torch.optim常用优化器介绍
## 1.1 torch.optim.SGD()
(1)算法原理：
![](./images/SGD的公式.png)

(2)参数介绍：
```python
torch.optim.SGD(params, lr, momentum, weight_decay) #仅仅包含了最常用的参数
```

1> momentum: 加速收敛，抑制震荡。通常设置为0-1之间。当设置之后，参数的更新方式就变成了下面的方式：
![](./images/SGD加上momentum的公式.png)

2> weight_decay:在loss函数上加上一个L2正则化项，通过惩罚大的参数值来避免过拟合。加上之后的公式和参数更新方式变为：
![](./images/SGD加上weightdecay的公式.png)
![](./images/SGD加上weightdecay的公式_1.png)

(3)缺点：
1>对于学习率十分敏感，通常需要花费大量的时间来微调学习率这个超参数；<br>
2>收敛的速度相对较慢，特别是处在损失函数地形的狭长的山谷中的时候，加上momentum之后会改善；并且容易陷入局部最优，特别是学习率特别小的时候；<br>

(4)适用情况：
因为每次都是在一个mini-batch上计算的梯度，因此十分节省内存，非常适合用于大数据集的训练。



## 1.2 torch.optim.Adam()
(1)定义：Adam（Adaptive Moment Estimation）结合了SGD with momentum和RMSprop的优点。

(2)算法原理：<br>
![](./images/Adam算法原理.jpg)

(3)参数介绍：
```python
    torch.optim.Adam(
        params,
        lr,
        betas = (0.9, 0.999), #对于beata 1 和 beta 2的衰减因子：Decay factors for momentum (β1) and adaptive scaling (β2)
        eps = 1e-8, #用于避免除以0的小值
        weight_decay = 0, #L2正则化
        amsgrad = False #是否使用AMSGrad，用于更好收敛的替换更新法则；通过强制执行非递减的二阶矩估计（non-decreasing second moment estimate）来解决Adam算法中的收敛问题。
    )
```

(4)适用情况：
1>通常用于小数据集，大的数据集一般使用SGD + momentum会更好；<br>
2>在RL或者NLP这种存在noisy gradients的任务中，更适用；

(5)改进的版本——AdamW
```python
    optimizer = torch.optim.AdamW(model.parameters(), lr = 0.001, weight_decay = 0.01)
    #AdamW修正了Adam错误的weight_decay方法；
    #更加适用于Transformer-based models，比如ViT或者BERT等
```

## 1.3 其他常用的优化器
![](./images/其他常用的优化器.jpg)